# Doppelpendel

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Am Beispiel eines Doppelpendels werden in diesem Jupyter-Notebook Aspekte der nichtlinearen Dynamik wie Integrabilität und chaotisches Verhalten untersucht. Insgesamt besteht dieses Jupyter-Notebook aus vier Teilen, auf die in mehreren Abschnitten des Buches Bezug genommen wird.

Zunächst wird mit den gleichen Methoden wie in den meisten anderen Jupyter-Notebooks zu Problemen der Mechanik die Lösung der Bewegungungsgleichungen des Doppelpendels untersucht, die in dimensionsloser Form durch

\begin{align}
 \ddot\phi_1 &= \frac{\alpha\sin\phi_2\cos\phi_2-\phi_1-\alpha\beta\dot\phi_2-(1+\alpha)\sin\phi_1 - \alpha \dot\phi_1\sin(\Delta\phi)}{1+\alpha\sin^2(\Delta \phi)}\\
 \ddot\phi_2 &= \frac{\dot\phi_1^2\sin(\Delta\phi)-\ddot\phi_1\cos(\Delta\phi)-\sin(\phi_2)}{\beta}
\end{align}

gegeben sind. Dabei ist $\Delta\phi=\phi_1-\phi_2$. Die Parameter $\alpha$ und $\beta$ geben das Verhältnis der beiden beteiligten Massen bzw. der beiden Fadenlängen an.

Im zweiten und dritten Teil wird auf die Untersuchung von nichtlinearer Dynamik mit Hilfe von Poincaré-Plots eingegangen. Dabei wird zunächst ein einzelner Startpunkt für die Dynamik betrachtet. Anschließend wird durch Verwendung mehrerer Startpunkte gezeigt, wie sich ein Gesamtbild für den Poincaré-Plot zusammensetzt.

Im vierten Teil wird schließlich der Ljapunov-Exponent als Maß für das chaotische Verhalten einer Dynamik berechnet.

## Ort und Geschwindigkeit als Funktion der Zeit

### Importanweisungen

In [ ]:
from math import cos, log, sin, sqrt
from scipy import integrate
import numpy as np
import numpy.linalg as LA
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementierung des Differentialgleichungssystems

Die beiden oben genannten Bewegungsgleichungen werden in vier Differentialgleichungen 1. Ordnung umgewandelt, um die Ausdrücke in der nachfolgenden Funktion zu erhalten.

In [ ]:
def dx_dt(t, x, alpha, beta):
    phi1, omega1, phi2, omega2 = x
    dphi = phi1-phi2
    a_phi1_numer = (-alpha*beta*omega2**2*sin(dphi)
                    - (1+alpha)*sin(phi1)
                    - alpha*omega1**2*sin(dphi)*cos(dphi)
                    + alpha*sin(phi2)*cos(dphi))
    a_phi1_denom = 1+alpha*sin(dphi)**2
    a_phi1 = a_phi1_numer / a_phi1_denom
    a_phi2 = (omega1**2*sin(dphi)
              - sin(phi2)
              - a_phi1*cos(dphi)) / beta
    return omega1, a_phi1, omega2, a_phi2

### Lösung des Differentialgleichungssystems

Die Bewegungsgleichungen werden mit Hilfe der SciPy-Funktion `integrate.solve_ivp` gelöst, wobei die Schranken für den absoluten und den relativen Fehler hier fest auf 10⁻¹² gesetzt werden. Der Lösungsvektor `solution.y` enthält die Zeitverläufe von $\phi_1, \dot\phi_1, \phi_2$ und $\dot\phi_2$.

In [ ]:
def trajectory(t_end, n_out, alpha, beta, phi1_0, omega1_0,
               phi2_0, omega2_0):
    t_values = np.linspace(0, t_end, n_out)
    x_0 = (phi1_0, omega1_0, phi2_0, omega2_0)
    solution = integrate.solve_ivp(dx_dt, (0, t_end), x_0,
                                   args=(alpha, beta),
                                   t_eval=t_values,
                                   atol=1e-12, rtol=1e-12)
    return t_values, solution.y

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit Hilfe der Schieberegler können die folgenden Parameter eingestellt werden:
- `t_end`: Länge des zu betrachtenden Zeitintervalls
- `n_out`: Anzahl der zu betrachtenden Zeitpunkte
- `alpha`: Verhältnis der Pendelmassen $m_2/m_1$
- `beta`: Verhältnis der Pendellängen $l_2/l_1$
- `phi1_0`: Anfangswinkel $\phi_1$
- `omega1_0`: anfängliche Winkelgeschwindigkeit $\dot\phi_1$
- `phi2_0`: Anfangswinkel $\phi_2$
- `omega2_0`: anfängliche Winkelgeschwindigkeit $\dot\phi_2$

Die Zeitverläufe von $\phi_1, \dot{\phi}_1, \phi_2$ und $\dot{\phi}_2$ werden graphisch dargestellt.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=50, min=0.1, max=200, step=0.1,
                   description=r"$t_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=500, min=10, max=2000, step=10,
                   description=r"$n_\text{out}$"),
               "alpha":
               widgets.FloatSlider(
                   value=1, min=0, max=2, step=0.1,
                   description=r"$\alpha$"),
               "beta":
               widgets.FloatSlider(
                   value=0.8, min=0, max=2, step=0.1,
                   description=r"$\beta$"),
               "phi1_0":
               widgets.FloatSlider(
                   value=0.09, min=0, max=1, step=0.01,
                   description=r"$\phi_1(0)$"),
               "omega1_0":
               widgets.FloatSlider(
                   value=0.16, min=0, max=1, step=0.01,
                   description=r"$\omega_1(0)$"),
               "phi2_0":
               widgets.FloatSlider(
                   value=0, min=-0.5, max=0.5, step=0.01,
                   description=r"$\phi_2(0)$"),
               "omega2_0":
               widgets.FloatSlider(
                   value=-2.58, min=-3, max=3, step=0.01,
                   description=r"$\omega_2(0)$")
               }

@interact(**widget_dict)
def plot_time_dependence(t_end, n_out, alpha, beta, phi1_0,
                         omega1_0, phi2_0, omega2_0):
    t_values, solution_y = trajectory(t_end, n_out,
                                      alpha, beta,
                                      phi1_0, omega1_0,
                                      phi2_0, omega2_0)
    (phi1_values, omega1_values,
     phi2_values, omega2_values) = solution_y

    fig, axs = plt.subplots(2, 2)
    axs[0, 0].plot(t_values, phi1_values)
    axs[0, 0].set_xlabel("$t$")
    axs[0, 0].set_ylabel(r"$\phi_1$")

    axs[0, 1].plot(t_values, phi2_values)
    axs[0, 1].set_xlabel("$t$")
    axs[0, 1].set_ylabel(r"$\phi_2$")

    axs[1, 0].plot(t_values, omega1_values)
    axs[1, 0].set_xlabel("$t$")
    axs[1, 0].set_ylabel(r"$\omega_1$")

    axs[1, 1].plot(t_values, omega2_values)
    axs[1, 1].set_xlabel("$t$")
    axs[1, 1].set_ylabel(r"$\omega_2$")

## Erzeugung des Poincaré-Plots für einen Startpunkt

### Abbruchbedingungen für $\dot\phi_1=0$

Der Poincaré-Schnitt soll durch die Bedingung $\dot\phi_1=0$ definiert sein, wobei die Winkelgeschwindigkeit beim Nulldurchgang zunehmen soll. Da die Integration später abschnittsweise erfolgen wird, definieren wir hier zwei Abbruchbedingungen, die zu Nulldurchgängen von $\dot\phi_1$ in zunehmender bzw. abnehmender Richtung gehören.

In [ ]:
def omega1_null_pos(t, x, alpha, beta):
    return x[1]

omega1_null_pos.terminal = True
omega1_null_pos.direction = 1

In [ ]:
def omega1_null_neg(t, x, alpha, beta):
    return x[1]

omega1_null_neg.terminal = True
omega1_null_neg.direction = -1

###  Berechnung der Punkte für den Poincaré-Plot

Die Integration der Bewegungsgleichungen erfolgt mit Hilfe der SciPy-Funktion `integrate.solve_ivp` stückweise zwischen Durchgängen durch die Hyperebene $\dot\phi_1=0$. Da im Poincaré-Schnitt nur Durchgänge aufgenommen werden dürfen, die in die gleiche Richtung erfolgen, werden die Werte von $\phi_1$ und $\phi_2$ nur gespeichert, wenn die durch `omega_null_pos` definierte Bedingung erfüllt ist.

In [ ]:
def poincare_points(dt, n_out, alpha, beta,
                    phi1_0, omega1_0, phi2_0, omega2_0):
    x_0 = (phi1_0, omega1_0, phi2_0, omega2_0)
    phi1_values = []
    phi2_values = []
    n = 0
    t_start = 0
    t_end = dt
    while n < n_out:
        for event, store in ((omega1_null_pos, True),
                             (omega1_null_neg, False)):
            solution = integrate.solve_ivp(
                dx_dt, (t_start, t_end), x_0,
                args=(alpha, beta),
                events=event, atol=1e-12, rtol=1e-12)
            if solution.t_events[0].size > 0:
                t_end = solution.t_events[0][0]
                if store:
                    phi1_values.append(solution.y[0][-1])
                    phi2_values.append(solution.y[2][-1])
                    n = n+1
            t_start = t_end
            t_end = t_start+dt
            x_0 = solution.y[:, -1]
    return phi1_values, phi2_values

### Implementierung der Bedienelemente und graphische Darstellung des Poincaré-Plots

Mit Hilfe der Schieberegler können die folgenden Parameter eingestellt werden:
- `dt`: Länge des Integrationsintervalls, in dem nach einem Durchgang durch die Hyperebene $\dot\phi_1=0$ gesucht werden soll
- `n_out`: Anzahl der Punkte im Poincaré-Plot
- `alpha`: Verhältnis der Pendelmassen $m_2/m_1$
- `beta`: Verhältnis der Pendellängen $l_2/l_1$
- `phi1_0`: Anfangswinkel $\phi_1$
- `omega1_0`: anfängliche Winkelgeschwindigkeit $\dot\phi_1$
- `phi2_0`: Anfangswinkel $\phi_2$
- `omega2_0`: anfängliche Winkelgeschwindigkeit $\dot\phi_2$

Die Darstellung des Poincaré-Plots erfolgt in der $\phi_1$-$\phi_2$-Ebene.

In [ ]:
widget_dict = {"dt":
               widgets.FloatSlider(
                   value=50, min=0.1, max=200, step=0.1,
                   description=r"$\Delta t$"),
               "n_out":
               widgets.IntSlider(
                   value=2000, min=10, max=100000, step=10,
                   description=r"$n_\text{out}$"),
               "alpha":
               widgets.FloatSlider(
                   value=1, min=0, max=2, step=0.1,
                   description=r"$\alpha$"),
               "beta":
               widgets.FloatSlider(
                   value=1, min=0, max=2, step=0.1,
                   description=r"$\beta$"),
               "phi1_0":
               widgets.FloatSlider(
                   value=0, min=-1, max=1, step=0.1,
                   description=r"$\phi_1(0)$"),
               "omega1_0":
               widgets.FloatSlider(
                   value=1, min=-1, max=1, step=0.1,
                   description=r"$\omega_1(0)$"),
               "phi2_0":
               widgets.FloatSlider(
                   value=1, min=0, max=2, step=0.1,
                   description=r"$\phi_2(0)$"),
               "omega2_0":
               widgets.FloatSlider(
                   value=0, min=0, max=2, step=0.1,
                   description=r"$\omega_2(0)$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_poincare(dt, n_out, alpha, beta, phi1_0, omega1_0,
                  phi2_0, omega2_0):
    phi1_values, phi2_values = poincare_points(
        dt, n_out, alpha, beta, phi1_0, omega1_0, phi2_0,
        omega2_0)

    fig, ax = plt.subplots()
    ax.plot(phi1_values, phi2_values, linestyle="None",
            marker="o", markersize=1)
    ax.set_xlabel(r"$\phi_1$")
    ax.set_ylabel(r"$\phi_2$")

## Erzeugung eines Poincaré-Plots mit mehreren Startpunkten

### Berechnung der Punkte für den Poincaré-Plot

Um einen besseren Überblick über die Struktur des Poincaré-Plots zu erhalten, werden ausgehend von einem ganzen Gitter von Startpunkten $\phi_1(0)$ und $\phi_2(0)$ die Durchstoßpunkte durch die Hyperebene $\dot\phi_1=0$ berechnet. Die Startpunkte werden äquidistant zwischen $-\phi_{1,\text{max}}$ und $\phi_{1,\text{max}}$ bzw. $-\phi_{2,\text{max}}$ und $\phi_{2,\text{max}}$ gewählt. Alle Startpunkte gehören zur gleichen Gesamtenergie $E_\text{ges}$, so sich mit der dritten Anfangsbedingung $\dot\phi_1(0)=0$ der Anfangswert für $\dot\phi_2(0)$ berechnen lässt, wobei das positive Vorzeichen gewählt wird. Startpunkte, für die sich die Bedingung an die Gesamtenergie nicht erfüllen lässt, werden ausgeschlossen. Für jeden Startpunkt wird die Funktion `poincare_points` aufgerufen, um die folgenden Punkte im Poincaré-Plot zu berechnen. Als Ergebnis werden zwei Listen `all_phi1_values` und `all_phi2_values` zurückgegeben, die wiederum Listen für die Punkte enthalten, die sich aus einem Startpunkt ergeben haben. Damit ist es möglich, die verschiedenen Punktefamilien farblich unterschiedlich darzustellen.

Da die Berechnung des Poincaré-Plots für mehrere Startpunkte relativ lange dauern kann, werden die bereits abgearbeiteten Startpunkte angezeigt. Zudem ist die maximale Zahl der Gitterpunkte je Winkel mit Hilfe der Variable `n_max` auf `2*n_max+1` gleich 9 festgelegt. Ist man bereit, mehr Rechenzeit zu investieren, so kann man den Wert von `n_max` in der Funktion `poincare_points_mult` angepassen. Die Rechenzeit wird allerdings quadratisch mit `n_max` ansteigen.

In [ ]:
def poincare_points_mult(dt, n_out, alpha, beta,
                         phi1_max, phi2_max, e_ges):
    all_phi1_values = []
    all_phi2_values = []
    n_max = 4
    for phi1_start in np.linspace(-phi1_max, phi1_max,
                                  2*n_max+1):
        for phi2_start in np.linspace(-phi2_max, phi2_max,
                                      2*n_max+1):
            e_pot = (-(1+alpha)*cos(phi1_start)
                     - alpha*beta*cos(phi2_start))
            e_kin = e_ges - e_pot
            omega1_start = 0
            if e_kin >= 0:
                omega2_start = sqrt(2*e_kin/alpha) / beta
                phi1_values, phi2_values = poincare_points(
                    dt, n_out, alpha, beta,
                    phi1_start, omega1_start,
                    phi2_start, omega2_start)
                all_phi1_values.append(phi1_values)
                all_phi2_values.append(phi2_values)
                print(phi1_start, phi2_start)
    return all_phi1_values, all_phi2_values

### Implementierung der Bedienelemente und graphische Darstellung des Poincaré-Plots

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `dt`: Länge des Integrationsintervalls, in dem nach einem Durchgang durch die Hyperebene $\dot\phi_1=0$ gesucht werden soll
- `n_out`: Anzahl der Punkte im Poincaré-Plot pro Startpunkt
- `alpha`: Verhältnis der Pendelmassen $m_2/m_1$
- `beta`: Verhältnis der Pendellängen $l_2/l_1$
- `phi1_max`: betragsmäßige Grenze für 9 Gitterpunkte des Startwerts $\phi_1(0)$
- `phi2_max`: betragsmäßige Grenze für 9 Gitterpunkte des Startwerts $\phi_2(0)$
- `e_ges`: dimensionslose Gesamtenergie

Punkte des Poincaré-Plots, die zu unterschiedlichen Startpunkten gehören, sind in verschiedenen Farben dargestellt.

In [ ]:
widget_dict = {"dt":
               widgets.FloatSlider(
                   value=50, min=0.1, max=200, step=0.1,
                   description=r"$\Delta t$"),
               "n_out":
               widgets.IntSlider(
                   value=100, min=100, max=10000, step=100,
                   description=r"$n_\text{out}$"),
               "alpha":
               widgets.FloatSlider(
                   value=1, min=0, max=2, step=0.1,
                   description=r"$\alpha$"),
               "beta":
               widgets.FloatSlider(
                   value=1, min=0, max=2, step=0.1,
                   description=r"$\beta$"),
               "phi1_max":
               widgets.FloatSlider(
                   value=2, min=0, max=2, step=0.1,
                   description=r"$\phi_{1,\text{max}}$"),
               "phi2_max":
               widgets.FloatSlider(
                   value=2, min=0, max=2, step=0.1,
                   description=r"$\phi_{2,\text{max}}$"),
               "e_ges":
               widgets.FloatSlider(
                   value=-1.5, min=-2, max=2, step=0.1,
                   description=r"$E_\text{ges}$")
               }

@interact_start(**widget_dict)
def plot_poincare_mult(dt, n_out, alpha, beta, phi1_max,
                       phi2_max, e_ges):
    phi1_list, phi2_list = poincare_points_mult(
        dt, n_out, alpha, beta, phi1_max, phi2_max, e_ges)

    fig, ax = plt.subplots()
    for phi1_values, phi2_values in zip(phi1_list,
                                        phi2_list):
        ax.plot(phi1_values, phi2_values, linestyle="None",
                marker="o", markersize=1)
    ax.set_xlabel(r"$\phi_1$")
    ax.set_ylabel(r"$\phi_2$")

## Ljapunov-Exponent

Im letzten Abschnitt soll eine Näherung für den Ljapunov-Exponenten

$$\lambda = \lim_{t \to \infty} \lim_{\Delta(0) \to 0} \frac{1}{t} \ln\left( \frac{\Delta(t)}{\Delta(0)} \right)$$

berechnet werden, wobei wir den Grenzwert $\Delta(0) \to 0$ durch einen sehr kleinen Anfangsabstand `delta_0` ersetzen, der durch die schrittweise Skalierung des Abstandes nach jedem Zeitschritt effektiv noch um viele Größenordnungen reduziert wird. Der Grenzwert $t \to \infty$ wird durch eine Zeit $t$ ersetzt, die in jedem Zeitschritt vergrößert wird, bis die Rechnung bei $t_\text{end}$ beendet wird.

### Berechnung des Ljapunov-Exponenten

Neben dem Startpunkt `x_0` mit den Komponenten $(\phi_1(0), \dot\phi_1(0), \phi_2(0), \dot\phi_2(0))$ definieren wir einen zweiten, um $\Delta(0)$ in Richtung von $\phi_1$ verschobenen Startpunkt `x_1`. Ausgehend von diesen Startpunkten wird die SciPy-Funktion `integrate.solve_ivp` benutzt, um die Trajektorie vom jeweiligen Startzeitpunkt `t_start` bis `t_start+delta_t` zu berechnen. Da uns nur die Phasenraumpunkte am Ende des Zeitintervalls interessieren, wird dem Argument `t_eval` ein Tupel übergeben, das nur die entsprechende Zeit enthält.

Aus den so erhaltenen Endpunkten berechnet man die neuen Startpunkte. `x_0` kann direkt übernommen werden, während `x_1` so in Richtung von `x_0` verschoben wird, dass der neue Abstand wieder gleich `delta_0` ist.

Aus dem Abstand `delta_1` der Phasenraumpunkte `x_0` und `x_1` und dem anfänglichen Abstand `delta_0` wird der Ljapunov-Exponent für das betreffende Zeitintervall berechnet. Anschließend wird noch der Mittelwert für den Ljapunov-Exponenten für den gesamten, bisher betrachteten Zeitraum berechnet. Neben den Startzeiten der Zeitintervalle wird eine Liste mit den, wie gerade beschrieben gemittelten Ljapunov-Exponenten übergeben.

In [ ]:
def ljapunov_exponent(t_end, n_out, alpha, beta,
                      phi1_0, omega1_0,
                      phi2_0, omega2_0, delta_0):
    x_0 = (phi1_0, omega1_0, phi2_0, omega2_0)
    x_1 = (phi1_0 + delta_0, omega1_0, phi2_0, omega2_0)
    ljapunov_values = []
    ljapunov_exp = 0
    t_start, delta_t = np.linspace(0, t_end, n_out+1,
                                   retstep=True)
    for n, (t0, t1) in enumerate(zip(t_start[:-1],
                                     t_start[1:])):
        solution = integrate.solve_ivp(
            dx_dt, (t0, t1), x_0, args=(alpha, beta),
            t_eval=(t1,), atol=1e-12, rtol=1e-12)
        x_0 = solution.y[:, 0]
        solution = integrate.solve_ivp(
            dx_dt, (t0, t1), x_1, args=(alpha, beta),
            t_eval=(t1,), atol=1e-12, rtol=1e-12)
        x_1 = solution.y[:, 0]
        delta_1 = LA.norm(x_1-x_0)
        x_1 = x_0 + (x_1-x_0) * delta_0/delta_1
        incr_ljapunov_exp = log(delta_1 / delta_0)
        incr_ljapunov_exp = incr_ljapunov_exp / delta_t
        ljapunov_exp = n*ljapunov_exp + incr_ljapunov_exp
        ljapunov_exp = ljapunov_exp / (n+1)
        ljapunov_values.append(ljapunov_exp)
    return t_start[:-1], ljapunov_values

### Implementierung der Eingabeelemente und graphische Darstellung des Ljapunov-Exponenten

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des zu betrachtenden Zeitintervalls
- `n_out`: Anzahl der zu betrachtenden Zeitpunkte
- `alpha`: Verhältnis der Pendelmassen $m_2/m_1$
- `beta`: Verhältnis der Pendellängen $l_2/l_1$
- `phi1_0`: Anfangswinkel $\phi_1(0)$
- `omega1_0`: anfängliche Winkelgeschwindigkeit $\dot\phi_1(0)$
- `phi2_0`: Anfangswinkel $\phi_2(0)$
- `omega2_0`: anfängliche Winkelgeschwindigkeit $\dot\phi_2(0)$
- `delta_0`: Anfangsabstand für $\phi_1(0)$

Die Voreinstellungen für die Anfangsbedingungen sind so gewählt, dass man sich im chaotischen Bereich befindet, weswegen der Ljapunov-Exponent positiv ist. Wenn man die Werte von `omega_1` und `phi_2` halbiert, befindet man sich in einer regulären Insel, so dass der Ljapunov-Exponent gegen null konvergiert.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=1000, min=0.1, max=2000, step=0.1,
                   description=r"$t_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=10000, min=10, max=20000, step=10,
                   description=r"$n_\text{out}$"),
               "alpha":
               widgets.FloatSlider(
                   value=1, min=0, max=2, step=0.1,
                   description=r"$\alpha$"),
               "beta":
               widgets.FloatSlider(
                   value=1., min=0, max=2, step=0.1,
                   description=r"$\beta$"),
               "phi1_0":
               widgets.FloatSlider(
                   value=0, min=0, max=2, step=0.1,
                   description=r"$\phi_1(0)$"),
               "omega1_0":
               widgets.FloatSlider(
                   value=1.6, min=0, max=2, step=0.1,
                   description=r"$\omega_1(0)$"),
               "phi2_0":
               widgets.FloatSlider(
                   value=0.8, min=0, max=2, step=0.1,
                   description=r"$\phi_2(0)$"),
               "omega2_0":
               widgets.FloatSlider(
                   value=0, min=0, max=2, step=0.1,
                   description=r"$\omega_2(0)$"),
               "delta_0":
               widgets.FloatLogSlider(
                   value=1e-6, min=-10, max=-2, step=0.1,
                   description=r"$\Delta_{0}$")
               }

@interact_start(**widget_dict)
def plot_ljapunov(t_end, n_out, alpha, beta, phi1_0,
                  omega1_0, phi2_0, omega2_0, delta_0):
    t_values, ljapunov_values = ljapunov_exponent(
        t_end, n_out, alpha, beta, phi1_0, omega1_0,
        phi2_0, omega2_0, delta_0)

    fig, ax = plt.subplots()
    ax.plot(t_values, ljapunov_values)
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\lambda$")
    ax.set_ylim(0, 0.5)